In [3]:
#Load Image and Annotations Function:
#Loads an image and its corresponding annotations from the JSON file.


In [ ]:
#Image Preprocessing Function:
#Resize and normalize images.

In [8]:
from PIL import Image
import numpy as np
import json
import os

def load_and_preprocess_data(img_folder, json_folder, img_width=224, img_height=224):
    images = []
    labels = []

    # List all files in the img_folder assuming the naming convention matches your description
    for img_file in sorted(os.listdir(img_folder)):
        # Extract the number from the image file name, assuming the format "image_1.png"
        img_number = img_file.split('_')[1].split('.')[0]
        
        # Construct the full file paths
        img_path = os.path.join(img_folder, img_file)
        label_path = os.path.join(json_folder, f"label_{img_number}.json")

        #print(f"Loading image: {img_path}")
        #print(f"Loading label: {label_path}")

        # Load and resize image using PIL
        try:
            with Image.open(img_path) as img:
                img = img.resize((img_width, img_height), Image.ANTIALIAS)
                img = np.array(img, dtype=np.float32) / 255.0  # Normalize image data to 0-1
                images.append(img)
        except IOError:
            print(f"Failed to load or process image at {img_path}")
            continue

        # Load labels
        if not os.path.exists(label_path):
            print(f"Label file not found at {label_path}")
            continue
        
        with open(label_path, 'r') as file:
            data = json.load(file)
            box_data = []
            for item in data['boxes']:
                if item['label'] == 'ball':
                    # Normalize coordinates
                    x = float(item['x']) / data['width']
                    y = float(item['y']) / data['height']
                    width = float(item['width']) / data['width']
                    height = float(item['height']) / data['height']
                    box_data.append([x * img_width, y * img_height, width * img_width, height * img_height])
            labels.append(box_data)

    return np.array(images), labels

# Usage
img_folder = '../images'
json_folder = '../json_labeling'
images, labels = load_and_preprocess_data(img_folder, json_folder)


C:\Users\hanso\AppData\Local\Temp\ipykernel_1248\2541950852.py:25: DeprecationWarning: ANTIALIAS is deprecated and will be removed in Pillow 10 (2023-07-01). Use LANCZOS or Resampling.LANCZOS instead.
  img = img.resize((img_width, img_height), Image.ANTIALIAS)


In [7]:
from PIL import Image, ImageDraw
import numpy as np

def display_image_with_boxes_pil(image, boxes, expand_by=4):
    """
    Displays an image with bounding boxes drawn on it using PIL, with each box slightly larger.
    Also converts the image to grayscale.

    Parameters:
    - image: The image array (numpy array).
    - boxes: A list of bounding boxes, each represented as [center_x, center_y, width, height].
    - expand_by: Number of pixels to increase the size of each box by (total, not per side).
    """
    # Ensure the image is in uint8 format
    if image.dtype != np.uint8:
        image = (255 * image).astype(np.uint8)

    # Convert numpy array image to PIL Image
    pil_img = Image.fromarray(image)
    pil_img = pil_img.convert('L')  # Convert to grayscale

    # Create a drawing context
    draw = ImageDraw.Draw(pil_img)

    # Draw each box
    for box in boxes:
        center_x, center_y, width, height = map(int, box)
        top_left = (center_x - (width // 2) - (expand_by // 2), center_y - (height // 2) - (expand_by // 2))
        bottom_right = (center_x + (width // 2) + (expand_by // 2), center_y + (height // 2) + (expand_by // 2))
        draw.rectangle([top_left, bottom_right], outline="white", width=2)  # White box on grayscale image

    # Display the image
    pil_img.show()

# Example usage (assuming images and labels are already loaded and preprocessed)
if len(images) > 0 and len(labels) > 0:
    display_image_with_boxes_pil(images[0], labels[0])

In [ ]:
#Data Augmentation Functions (optional):
#Rotate, flip, or crop images to create more diverse training data.
# NOT IMPLEMENTED

In [ ]:
#Convolution Function:
#Applies a convolution operation to an image.

In [13]:
import numpy as np

def convolve2d(image, kernel, stride=1, padding=0):
    """
    Apply a 2D convolution operation to an image.
    
    Args:
    image (np.array): The input image array (height, width, channels).
    kernel (np.array): The kernel (filter) to apply (height, width).
    stride (int): The stride of the convolution.
    padding (int): The amount of padding around the image.
    
    Returns:
    np.array: The convolved image.
    """
    kernel_height, kernel_width = kernel.shape
    image_height, image_width, num_channels = image.shape

    # Add padding to the image
    padded_height = image_height + 2 * padding
    padded_width = image_width + 2 * padding
    padded_image = np.zeros((padded_height, padded_width, num_channels))
    padded_image[padding:padding+image_height, padding:padding+image_width] = image

    # Determine the output dimensions
    out_height = ((padded_height - kernel_height) // stride) + 1
    out_width = ((padded_width - kernel_width) // stride) + 1
    output = np.zeros((out_height, out_width, num_channels))

    # Perform the convolution operation
    for z in range(num_channels):  # Iterate over each channel
        for y in range(out_height):
            for x in range(out_width):
                output[y, x, z] = np.sum(
                    kernel * padded_image[
                        y * stride : y * stride + kernel_height,
                        x * stride : x * stride + kernel_width,
                        z
                    ]
                )
    return output





In [14]:
# Example usage
from PIL import Image

# Load an image and convert to a numpy array
img = Image.open('../images/image_1.png')
img_np = np.array(img)

# Define a simple edge-detection kernel
kernel = np.array([[1, 0, -1],
                   [1, 0, -1],
                   [1, 0, -1]])

# Apply the convolution operation
convolved_image = convolve2d(img_np, kernel, stride=1, padding=1)


In [16]:
import matplotlib.pyplot as plt
import numpy as np

def visualize_images(original_img, convolved_img):
    """
    Visualize the original and convolved images side by side.

    Args:
    original_img (np.array): The original image.
    convolved_img (np.array): The image after convolution.
    """
    # Normalize the convolved image for better visualization
    convolved_img_normalized = (convolved_img - convolved_img.min()) / (convolved_img.max() - convolved_img.min())

    # Plot the original and the convolved images
    fig, axes = plt.subplots(1, 2, figsize=(12, 6))
    axes[0].imshow(original_img, cmap='gray')
    axes[0].set_title('Original Image')
    axes[0].axis('off')

    axes[1].imshow(convolved_img_normalized, cmap='gray')
    axes[1].set_title('Convolved Image')
    axes[1].axis('off')

    plt.show()


In [17]:
from PIL import Image
import numpy as np

# Load an image and convert to a numpy array
img_path = '../images/image_1.png'  # Change this to the path of your image
img = Image.open(img_path).convert('L')  # Convert to grayscale for simplicity
img_np = np.array(img)

# Define a simple edge-detection kernel
kernel = np.array([[1, 0, -1],
                   [1, 0, -1],
                   [1, 0, -1]])

# Apply the convolution operation
convolved_image = convolve2d(img_np, kernel, stride=1, padding=1)

# Visualize the original and the convolved images
visualize_images(img_np, convolved_image)


FileNotFoundError: [Errno 2] No such file or directory: 'path_to_your_image.jpg'

In [ ]:
#ReLU Activation Function:
#Implements the ReLU activation to introduce non-linearity.

In [ ]:
#Pooling Function (optional but recommended):
#Applies a max pooling operation to reduce spatial dimensions.

In [ ]:
#Flatten Function:
#Converts multi-dimensional arrays into a 1D vector for the fully connected layer.

In [ ]:
#Fully Connected Layer Function:
#Performs linear transformation for classification or coordinate prediction.

In [ ]:
#Loss Function:
#Computes the error between predicted and actual bounding boxes (e.g., Mean Squared Error).

In [ ]:
#Optimizer Function:
#Adjusts model parameters based on the gradients of the loss function (e.g., SGD).

In [ ]:
#Training Loop Function:
#Manages the batch processing of images and annotations, forward pass, loss computation, and parameter updates.

In [ ]:
#Model Evaluation Function:
#Assesses the performance of the model on a validation set.

In [ ]:
#Bounding Box Prediction Function:
#Interprets the network output to predict bounding box coordinates.

In [ ]:
#Utility Functions:
#Includes functions for initializing weights, handling data shuffling, and converting coordinates.